Numerical simulation of direct dynamics

In [8]:
import sympy as sym
from sympy import Symbol, symbols, cos, sin, Matrix, simplify
from sympy.vector import CoordSys3D
from sympy.physics.mechanics import dynamicsymbols, mlatex, init_vprinting
init_vprinting()
from IPython.display import display, Math

eq = lambda lhs, rhs: display(Math(lhs + '=' + mlatex(rhs)))
eq = lambda lhs, rhs: display(Math(r'\begin{array}{l l}' + lhs +
                                   '&=&' + mlatex(rhs) + r'\end{array}'))

from minjerk import minjerk
from invkin2_2d import invkin


ModuleNotFoundError: No module named 'minjerk'

$$
\ddot{q} \quad=\quad M(q)^{-1} \left[\tau - C(q,\dot{q}) - G(q) - E(q,\dot{q}) \right]
\label{}
$$


$$
\left\{
\begin{array}{l l}
\dfrac{\mathrm{d} q}{\mathrm{d}t} &amp;=&amp; \dot{q}, \quad &amp;q(t_0) = q_0
\\
\dfrac{\mathrm{d} \dot{q}}{\mathrm{d}t} &amp;=&amp; M(q)^{-1} \left[\tau - C(q,\dot{q}) - G(q) - E(q,\dot{q}) \right], \quad &amp;\dot{q}(t_0) = \dot{q}_0
\end{array}
\right.
\label{}
$$

In [9]:
height, mass = 1.70,               70  # m, kg
L1n, L2n     = 0.188*height,       0.253*height
d1n, d2n     = 0.436*L1n,          0.682*L2n
m1n, m2n     = 0.0280*mass,        0.0220*mass
rg1n, rg2n   = 0.322,              0.468
I1n, I2n     = m1n*(rg1n*L1n)**2,  m2n*(rg2n*L2n)**2
T1a = 72
T2a = 30

duration = 4

xi, yi = 0, -L1n-L2n
xf, yf = L1n, L2n
gn = 9.81  # gravity acceleration m/s2

In [10]:

time, rlin, vlin, alin, jlin = minjerk([xi, yi], [xf, yf], duration=duration)

rang = invkin(time, rlin, L1=L1n, L2=L2n)

NameError: name 'minjerk' is not defined

In [13]:
def diff_c(rang, duration):
    """Numerical differentiations using the central difference for the angular data.
    """
    # central difference (f(x+h)-f(x-h))/(2*h)
    dt = duration/(ang.shape[0]-1)
    vang = np.empty_like(rang)
    vang[:, 0] = np.gradient(rang[:, 0], dt)
    vang[:, 1] = np.gradient(rang[:, 1], dt)
      
    return vang

vang = diff_c(rang, duration)

NameError: name 'rang' is not defined

In [16]:
def dyna(time, L1n, L2n, d1n, d2n, m1n, m2n, gn, I1n, I2n, q1, q2, rang, vang, Fexn, Feyn, M, C, G, E):
    """Numerical calculation and plot for the torques of a planar two-link system.
    """
    from sympy import lambdify, symbols
    
    Mfun  = lambdify((I1, I2, L1, L2, d1, d2, m1, m2, q1, q2), M, 'numpy')
    Mn    = Mfun(I1n, I2n, L1n, L2n, d1n, d2n, m1n, m2n, rang[:, 0], rang[:, 1])
    M00   = Mn[0, 0]
    M01   = Mn[0, 1]
    M10   = Mn[1, 0]
    M11   = Mn[1, 1]
    Q1d, Q2d = symbols('Q1d Q2d')
    dicti = {q1.diff(t, 1):Q1d, q2.diff(t, 1):Q2d}
    C0fun = lambdify((L1, d2, m2, q2, Q1d, Q2d), C[0].subs(dicti), 'numpy')
    C0    = C0fun(L1n, d2n, m2n, rang[:, 1], vang[:, 0], vang[:, 1])
    C1fun = lambdify((L1, d2, m2, q2, Q1d, Q2d), C[1].subs(dicti), 'numpy')
    C1    = C1fun(L1n, d2n, m2n, rang[:, 1], vang[:, 0], vang[:, 1])
    G0fun = lambdify((L1, d1, d2, m1, m2, g, q1, q2), G[0], 'numpy')
    G0    = G0fun(L1n, d1n, d2n, m1n, m2n, gn, rang[:, 0], rang[:, 1])
    G1fun = lambdify((L1, d1, d2, m1, m2, g, q1, q2), G[1], 'numpy')
    G1    = G1fun(L1n, d1n, d2n, m1n, m2n, gn, rang[:, 0], rang[:, 1])
    E0fun = lambdify((L1, L2, q1, q2, Fex, Fey), E[0], 'numpy')
    E0    = E0fun(L1n, L2n, rang[:, 0], rang[:, 1], 0, 0)
    E1fun = lambdify((L1, L2, q1, q2, Fex, Fey), E[1], 'numpy')
    E1    = E1fun(L1n, L2n, rang[:, 0], rang[:, 1], Fexn, Feyn)
          
    return M00, M01, M10, M11, C0, C1, G0, G1, E0, E1


In [17]:
Fexn, Feyn = 0, 0
M00, M01, M10, M11, C0, C1, G0, G1, E0, E1 = dyna(time, L1n, L2n, d1n, d2n, m1n, m2n, gn, I1n, I2n,
                                                  q1, q2, rang, vang, Fexn, Feyn, M, C, G, E)
acc1 = (T1a-C0-G0-E0)/(M00+M01)
acc2 = (T2a-C1-G1-E1)/(M10+M11)

NameError: name 'time' is not defined